In [67]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import re

def reset_timestamp(df):
    df['timestamp'] = df['timestamp'] - df['timestamp'][0]
    return df

def process_line(line):
    timestamp = line.split(' ')[1]
    content = line.split('  : ')[1]
#     print(content)
    pairs = content.split(',')
#     print(pairs)
    d = {}
    cpugpu = re.search("CPU4/GPU=\d+/\d+,\d+/\d+", content).group()
    cpugpu = cpugpu.split('=')[1]
    cpugpu = cpugpu.split('/')
    d['CPU Level'] = int(cpugpu[0])
    d['GPU Level'] = int(cpugpu[1].split(',')[0])
    d['CPU Frequency (MHz)'] = int(cpugpu[1].split(',')[1])
    d['GPU Frequency (MHz)'] = int(cpugpu[2])
    for x in pairs:
        tmp = x.split('=')
        if tmp[0] == 'CPU4/GPU' or len(tmp) < 2:
            continue
        if tmp[0] == 'FPS':
            d[tmp[0]] = int(tmp[1].split('/')[0])
        elif tmp[0] == 'Free':
            d[tmp[0]] = int(tmp[1][:-2])
        else:
            d[tmp[0]] = tmp[1].strip()
#     print(pairs[0].split('='))
#     d = {x.split('=')[0] : x.split('=')[1] for x in pairs}
#     print(d)
    d["timestamp"] = timestamp
#     print(d)
    return pd.DataFrame(d, index=[0])

def add_to_dataframe(df, timestamp, key, value):
    if key not in df.columns:
        df[key] = np.nan
    df.loc[timestamp, key] = value
    return df

def create_dataframe(file_path):
    df = pd.DataFrame()
    for line in open(file_path, 'r', encoding='utf-16'):
        if len(line) < 300:
            continue
        new_d = process_line(line)
        df = df.append(new_d)

#     df.index.name = 'timestamp'
#     df = df.reset_index()

    df['device'] = file_path.split('\\')[1]
    df['app'] = file_path.split('\\')[2].strip('-W')
    df['app_details'] = file_path.split('\\')[3]
    df['is_record'] = 'record' in file_path.split('\\')[4]
    df['is_wireless'] = '-W' in file_path.split('\\')[2]
    df['repetition'] = file_path.split('\\')[4].strip('record') if 'record' in file_path.split('\\')[4] else file_path.split('\\')[4].strip('replay')

    df.to_csv(f'.\\{file_path.strip(".log")}.csv', index=False)

    return df

def process_logcat():
    df_aggregated = pd.DataFrame()
    for root, dirs, files in os.walk(".", topdown=False):
        for name in files:
            if name == 'logcat_VrApi.log':
                print(os.path.join(root, name))
                df_aggregated = pd.concat([df_aggregated, create_dataframe(os.path.join(root, name))]) 
    # df_aggregated
    return df_aggregated


In [68]:
import warnings
warnings.filterwarnings("ignore")

df = process_logcat()
df

.\Quest-2\BeatSaber\PopStars-Easy\record0\logcat_VrApi.log
.\Quest-2\BeatSaber\PopStars-Easy\replay0.0\logcat_VrApi.log
.\Quest-2\BeatSaber\PopStars-Easy\replay0.1\logcat_VrApi.log
.\Quest-2\BeatSaber\PopStars-Easy\replay0.2\logcat_VrApi.log
.\Quest-2\BeatSaber\PopStars-Hard\record0\logcat_VrApi.log
.\Quest-2\BeatSaber\PopStars-Hard\replay0.0\logcat_VrApi.log
.\Quest-2\BeatSaber\PopStars-Hard\replay0.1\logcat_VrApi.log
.\Quest-2\BeatSaber\PopStars-Hard\replay0.2\logcat_VrApi.log
.\Quest-2\BeatSaber\PopStars-Medium\record0\logcat_VrApi.log
.\Quest-2\BeatSaber\PopStars-Medium\record1\logcat_VrApi.log
.\Quest-2\BeatSaber\PopStars-Medium\record2\logcat_VrApi.log
.\Quest-2\BeatSaber\PopStars-Medium\record3\logcat_VrApi.log
.\Quest-2\BeatSaber\PopStars-Medium\record4\logcat_VrApi.log
.\Quest-2\BeatSaber\PopStars-Medium\replay0.0\logcat_VrApi.log
.\Quest-2\BeatSaber\PopStars-Medium\replay0.1\logcat_VrApi.log
.\Quest-2\BeatSaber\PopStars-Medium\replay0.2\logcat_VrApi.log
.\Quest-2\BeatSaber-W\

PermissionError: [Errno 13] Permission denied: '.\\\\Quest-Pro\\BeatSaber\\PopStars-Medium\\replay0.2\\logcat_VrApi.csv'

In [ ]:
df[["FPS", "CPU Level", "CPU Frequency (MHz)", "Free"]] = df[["FPS", "CPU Level", "CPU Frequency (MHz)", "Free"]].apply(pd.to_numeric)

sns.boxplot(data=df, x='device', y='FPS', showmeans=True, meanline=True)
plt.show()

sns.boxplot(data=df, x='device', y='CPU Frequency (MHz)', showmeans=True, meanline=True)
plt.show()

sns.boxplot(data=df, x='device', y='CPU Level', showmeans=True, meanline=True)
plt.show()

sns.boxplot(data=df, x='app_details', y='FPS', showmeans=True, meanline=True)
plt.show()

sns.boxplot(data=df, x='device', y='Free', showmeans=True, meanline=True)
plt.show()
